In [ ]:
# Packages needed:
#   StatsBase
#   PyPlot
include("example_common.jl");

In [ ]:
const l = 1  # left node
const c = 4  # center node
const r = 7  # right node

# Column-stochastic random walk matrix
P1 = zeros(Float64, r, r)
for i = (l + 1):(r - 1)
    P1[[i - 1, i + 1], i] = 1 / 2
end
P1[[l + 1, c - 1, c], l] = [1 / 2, 1 / 6, 1 / 3]
P1[[r - 1, c + 1, c], r] = [1 / 2, 1 / 6, 1 / 3]

# Simulate the first-order Markov chain, starting at node 1
starting_node = c
(counts1, hist1) = sim_FOMC(P1, starting_node)
dist1 = normalize(counts1, 1)
dist1, hist1[1:50]

In [ ]:
# Nonbacktracking random walk transition probability tensor
P2 = zeros(Float64, r, r, r)
for i = (l + 1):(r - 1)
    P2[i + 1, i, i - 1] = 1.0
    P2[i - 1, i, i + 1] = 1.0
end
P2[[c, c + 1], r, r - 1] = [2 / 3, 1 / 3]
P2[[c, c - 1], l, l + 1] = [2 / 3, 1 / 3]
P2[[l + 1, c], c - 1, l] = [1 / 2, 1 / 2]
P2[[c, r - 1], c + 1, r] = [1 / 2, 1 / 2]
P2[[c + 1, c - 1], c, l] = [1 / 2, 1 / 2]
P2[[c + 1, c - 1], c, r] = [1 / 2, 1 / 2]

# Simulate the second-order Markov chain
starting_pair = (c, c - 1)
(counts2, hist2) = sim_SOMC(P2, starting_pair)
dist2 = normalize(counts2, 1)
dist2, hist2[1:50]

In [ ]:
# Spacey random walks!

# Simulate several spacey random walks
(counts3a, hist3a) = sim_SOSRW(P2, c, ones(Float64,r)/r) # uniform dist.
(counts3b, hist3b) = sim_SOSRW(P2, c, dist1)             # RW stat. dist.
(counts3c, hist3c) = sim_SOSRW(P2, c, dist2)             # NBRW stat. dist.
(counts3s, hist3s) = sim_SOSRW(P2, c, Float64[], true)   # super spacey

dist3a = normalize(counts3a, 1)
dist3b = normalize(counts3b, 1)
dist3c = normalize(counts3c, 1)
dist3s = normalize(counts3s, 1)
;

In [ ]:
# Let's check that we are actually getting out eigenvectors!

# First, fill in the zero columns.
P2a = copy(P2)
P2b = copy(P2)
P2c = copy(P2)
for j = 1:r, k = 1:r
    col = vec(P2[:, j, k])
    if maximum(col) == 0.0
        P2a[:, j, k] = ones(Float64,r)/r
        P2b[:, j, k] = dist1
        P2c[:, j, k] = dist2
    end
end

ya = apply(P2a, dist3a)
yb = apply(P2b, dist3b)
yc = apply(P2c, dist3c)

@show minimum(ya ./ dist3a), maximum(ya ./ dist3a)
@show minimum(yb ./ dist3b), maximum(yb ./ dist3b)
@show minimum(yc ./ dist3c), maximum(yc ./ dist3c)
;

In [ ]:
using PyPlot
plot(l:r, dist1,  lw=1, ls="-",  label="RW")
plot(l:r, dist2,  lw=1, ls="-",  label="NBRW")
plot(l:r, dist3a, lw=2, ls="--", label="SRW (uniform)")
plot(l:r, dist3b, lw=2, ls="--", label="SRW (RW stat. dist.)")
plot(l:r, dist3c, lw=2, ls="--", label="SRW (NBRW stat. dist.)")
plot(l:r, dist3s, lw=2, ls=":",  label="Super SRW")

xlabel("state")
ylabel("probability")

# Set legend on the outside
ax = gca()
box = ax[:get_position]()
ax[:set_position]([box[:x0], box[:y0], box[:width] * 0.8, box[:height]])
ax[:legend](loc="center left", bbox_to_anchor=(1, 0.75))

show()

In [ ]:
# Done!